# Make a full overview of number counts for SPIRE bands



In [1]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib as mpl
mpl.use('pdf')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np
#plt.rc('figure', figsize=(10, 6))
from matplotlib_venn import venn3

import herschelhelp 
from herschelhelp.utils import clean_table

from astropy.table import Table, vstack

import pyvo as vo

from pymoc import MOC

import time

import yaml



import warnings
warnings.filterwarnings('ignore')

In [2]:
#Then we establish the VO connection to our database
service = vo.dal.TAPService("https://herschel-vos.phys.sussex.ac.uk/__system__/tap/run/tap"
                           )

In [3]:
fields = yaml.load(open('../../../dmu2/meta_main.yml', 'r'))['fields']

In [4]:
bands = [
    'pacs_green', 
    'pacs_red', 
    'spire_250', 
    'spire_350', 
    'spire_500'
]
mag_tables = {}


for band in bands:
    mag_tables[band] = {}
    for f in fields:
      
        mag_tables[band].update({f['name']: None})

In [34]:
start = time.time()
for band in bands:
    print(band)
    
    for f in fields:
     
     
        try:
            
            mag_tables[band].update({f['name'] : Table.read('./data/{}_{}.fits'.format(band, f['name']))})
            print("loaded {} from file ({} objects)".format(band, len(mag_tables[band][f['name']])))
            continue
        except FileNotFoundError:
            print("Querying VOX for {} mags on {}".format(band, f['name']))
        try:
            query = """
            SELECT 
            f_{}
            FROM help_a_list.main
            WHERE help_a_list.main.f_{} IS NOT NULL
            AND help_a_list.main.field='{}'""".format(
            band, band, 
            f['name'].replace('Lockman-SWIRE','Lockman SWIRE' ).replace('HATLAS-NGP','NGP' ))
            job = service.submit_job(query, maxrec=100000000)
            job.run()
            job_url = job.url
            job.wait(phases='COMPLETED')
            #wait *=2

            print(job.phase)
        
            result = job.fetch_result()
            mag_tables[band].update({ f['name']: result.table})
            if len(mag_tables[band][f['name']]) != 0:
                print("Band {} field {} done in {} seconds with {} objects".format(
                    band, 
                    f['name'],                                                         
                    round(time.time() - start_time), 
                    len(mag_tables[band][f['name']])
                ))
        except:
            print('No mags.')
    
print("Total time: {} seconds".format(round(time.time() - start)))

pacs_green
Querying VOX for pacs_green mags on AKARI-NEP
COMPLETED
Band pacs_green field AKARI-NEP done in 78020 seconds with 14170 objects
Querying VOX for pacs_green mags on AKARI-SEP
COMPLETED
Band pacs_green field AKARI-SEP done in 78026 seconds with 63436 objects
Querying VOX for pacs_green mags on Bootes
COMPLETED
Band pacs_green field Bootes done in 78049 seconds with 471516 objects
Querying VOX for pacs_green mags on CDFS-SWIRE
COMPLETED
Band pacs_green field CDFS-SWIRE done in 78057 seconds with 74543 objects
Querying VOX for pacs_green mags on COSMOS
COMPLETED
Band pacs_green field COSMOS done in 78071 seconds with 320092 objects
Querying VOX for pacs_green mags on EGS
COMPLETED
Band pacs_green field EGS done in 78081 seconds with 177141 objects
Querying VOX for pacs_green mags on ELAIS-N1
COMPLETED
Band pacs_green field ELAIS-N1 done in 78096 seconds with 269561 objects
Querying VOX for pacs_green mags on ELAIS-N2
COMPLETED
Band pacs_green field ELAIS-N2 done in 78103 second

COMPLETED
Band spire_350 field ELAIS-S1 done in 79530 seconds with 194276 objects
Querying VOX for spire_350 mags on GAMA-09
COMPLETED
Band spire_350 field GAMA-09 done in 79591 seconds with 1386659 objects
Querying VOX for spire_350 mags on GAMA-12
COMPLETED
Band spire_350 field GAMA-12 done in 79646 seconds with 1099477 objects
Querying VOX for spire_350 mags on GAMA-15
COMPLETED
Band spire_350 field GAMA-15 done in 79705 seconds with 1236395 objects
Querying VOX for spire_350 mags on HDF-N
COMPLETED
Band spire_350 field HDF-N done in 79710 seconds with 834 objects
Querying VOX for spire_350 mags on Herschel-Stripe-82
No mags.
Querying VOX for spire_350 mags on Lockman-SWIRE
COMPLETED
Band spire_350 field Lockman-SWIRE done in 79783 seconds with 242065 objects
Querying VOX for spire_350 mags on HATLAS-NGP
COMPLETED
Band spire_350 field HATLAS-NGP done in 79855 seconds with 1233547 objects
Querying VOX for spire_350 mags on SA13
COMPLETED
Band spire_350 field SA13 done in 79858 second

In [31]:
mag_tables['spire_500']['COSMOS']['f_spire_500']

TypeError: 'NoneType' object is not subscriptable

In [35]:
mag_tables

{'pacs_green': {'AKARI-NEP': <Table masked=True length=14170>
  f_pacs_green
      uJy     
    float64   
  ------------
     35881.918
      262799.5
     506305.94
      55934.47
      68519.37
      65943.25
     497742.78
     157622.58
     493931.94
           ...
      43054.79
      50444.18
      86582.92
      76107.28
     420811.06
      474482.6
      367657.4
     80028.234
      87345.22
     41699.293,
  'AKARI-SEP': <Table masked=True length=63436>
  f_pacs_green
      uJy     
    float64   
  ------------
     500412.56
      507839.7
     500949.47
     491666.88
      514456.9
     505372.94
     505045.53
      504014.3
      504531.2
           ...
      499604.5
      493160.8
      514686.6
     500488.94
      493274.8
      489623.5
      493920.1
     493789.25
      500528.0
      505066.1,
  'Bootes': <Table masked=True length=471516>
  f_pacs_green
      uJy     
    float64   
  ------------
     500430.16
      498827.2
     500827.47
     524076.28
  

In [ ]:
#service.maxrec=100000000
print(service.hardlimit)

Herschel Stripe 82 hits the hard limit on rows so we must get it from two queries

In [36]:
write = True
read = False
for band in bands:
    for f in fields:
        if write and (len(mag_tables[band][f['name']]) != 0):
            clean_table(mag_tables[band][f['name']]).write(
                './data/{}_{}.fits'.format(band, f['name']), overwrite = True
            )
            print('Table cleaned and written to ./data/{}_{}.fits'.format(band, f['name']))
        elif read:
            mag_tables[band].update( 
                {f['name'] : Table.read('./data/{}_{}.fits'.format(band, f['name']))} 
            ) 
    

Table cleaned and written to ./data/pacs_green_AKARI-NEP.fits
Table cleaned and written to ./data/pacs_green_AKARI-SEP.fits
Table cleaned and written to ./data/pacs_green_Bootes.fits
Table cleaned and written to ./data/pacs_green_CDFS-SWIRE.fits
Table cleaned and written to ./data/pacs_green_COSMOS.fits
Table cleaned and written to ./data/pacs_green_EGS.fits
Table cleaned and written to ./data/pacs_green_ELAIS-N1.fits
Table cleaned and written to ./data/pacs_green_ELAIS-N2.fits
Table cleaned and written to ./data/pacs_green_ELAIS-S1.fits
Table cleaned and written to ./data/pacs_green_GAMA-12.fits
Table cleaned and written to ./data/pacs_green_GAMA-15.fits
Table cleaned and written to ./data/pacs_green_Lockman-SWIRE.fits
Table cleaned and written to ./data/pacs_green_HATLAS-NGP.fits
Table cleaned and written to ./data/pacs_green_XMM-LSS.fits
Table cleaned and written to ./data/pacs_red_AKARI-NEP.fits
Table cleaned and written to ./data/pacs_red_AKARI-SEP.fits
Table cleaned and written t

TypeError: object of type 'NoneType' has no len()

## Plot the histograms

In [ ]:
bands_plotting = {
    'pacs_green':['PACS Green','g'], 
    'pacs_red':['PACS red','r'], 
    'spire_250':['SPIRE 250','b'],  
    'spire_350':['SPIRE 350','c'],  
    'spire_500':['SPIRE 500','m'],  

}

In [ ]:
fig, ax = plt.subplots()


f = 'COSMOS'
area = 5
for band in [b for b in mag_tables if b.endswith('green')]:
    mask = np.isfinite(mag_tables[band][f]['f_'+band])
    mags = mag_tables[band][f][mask]['f_'+band]
    if not np.sum(mask)==0:
        #vz.hist(table[name][mask], bins='scott', label=label, alpha=.5)
        h = np.histogram(np.log10(mags), bins = 100)
        bin_width = (np.abs(h[1][5] - h[1][4]) )
        #ax.fill_between( h[1][:-1], h[0]/bin_width)#, alpha=0.4)
        ax.plot( h[1][:-1], h[0]/bin_width , c=bands_plotting[band][1])#, alpha=0.4)


ax.legend(loc=1, fontsize=8)

plt.xlim(2.,5.)
plt.xlabel("Log(Flux /[uJy])")
plt.yscale('log')
#plt.xscale('log')
#plt.ylim(0.,0.4)
plt.ylabel('Number [dex$^{-1}$]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)


#plt.savefig('./figs/numbers_g_en1.pdf', bbox_inches='tight')
#plt.savefig('./figs/numbers_g_en1.png', bbox_inches='tight')

In [ ]:
for n, f in enumerate(fields):
    print(f['name'])

In [ ]:
areas = {}
for band in bands:
    areas.update({band: {}})
    for n, f in enumerate(fields):
        f = f['name']
        f_moc = MOC(filename='../../../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(f))
    
        #band_moc = MOC(10,
        #    depth_result[~np.isnan(depth_result['ferr_{}_mean'.format(band)])]['hp_idx_o_10']
        #)
        area = f_moc.area_sq_deg #.flattened(order=10)
        areas[band].update({f: area})
        
np.save('./data/areas.npy', areas) 

In [ ]:

areas = np.load('./data/areas.npy').item()

In [ ]:
areas

In [ ]:
dim = [4,6]
fig, axes = plt.subplots(dim[1], dim[0], sharex=True, sharey=True)
plt.rcParams.update({'font.size': 12})

#area_per_pixel  = MOC(10, (1234)).area_sq_degrees
for n, f in enumerate(fields):
    f = f['name']
        
    x, y = np.floor_divide(n, dim[0]), np.remainder(n, dim[0])
    
   
    
    
    for band in mag_tables: #[b for b in mag_tables if b.endswith('g')]:
        mask = np.isfinite(mag_tables[band][f]['f_'+band])
        #mask &= (bands[band][0]['field'] == f)

        
        area = areas[band][f]
        #area=f_moc.area_sq_deg
        mags = mag_tables[band][f][mask]['f_'+band]
        if not np.sum(mask)==0:
            #vz.hist(table[name][mask], bins='scott', label=label, alpha=.5)
            h = np.histogram(np.log10(mags), bins = 100)
            bin_width = (np.abs(h[1][5] - h[1][4]) )
            #ax.fill_between( h[1][:-1], h[0]/bin_width)#, alpha=0.4)
            axes[x,y].plot( h[1][:-1], h[0]/(bin_width*area), c=bands_plotting[band][1])#, alpha=0.4)


 
   


    axes[x,y].get_xaxis().set_tick_params(direction='out')
    axes[x,y].xaxis.set_ticks_position('bottom')
    axes[x,y].tick_params(axis='x', labelsize=8)
    axes[x,y].set_xlim(2, 5)
    axes[x,y].set_xticks([2,3,4])
    axes[x,y].set_ylim(1.e1, 1.e6)
    axes[x,y].set_yscale('log')
    
    axes[x,y].scatter([-99],[-99], 
                      label=f, 
                      c='w', s=0.0001)
    axes[x,y].legend(frameon=False, loc=(-0.2, 0.8)) #, bbox_to_anchor=(0.1, 0.7, 0.2, 0.2)




for band in mag_tables: #[b for b in mag_tables if b.endswith('g')]:
    axes[dim[1]-1,dim[0]-1].scatter([-99],[-99], 
                      label=bands_plotting[band][0], 
                      c=bands_plotting[band][1], s=5.)
axes[dim[1]-1,dim[0]-1].legend( prop={'size': 10},ncol=1)    
    
axes[dim[1]-1,dim[0]-1].tick_params(axis='x', labelsize=8)
#axes[dim[1]-1,dim[0]-1].set_xlabel('band')
    
fig.text(0.5, 0.07, 'Log10(flux/[$\mu$Jy]) ', ha='center')
fig.text(0.04, 0.5, 'Differential $Herschel$ number counts [deg.$^{-2}$ dex$^{-1}$]', va='center', rotation='vertical')

fig.set_size_inches(10, 12)
fig.subplots_adjust(hspace=0, wspace=0)

plt.rc('axes', labelsize=12)
plt.savefig('./figs/numbers_Herschel_allfields.pdf', bbox_inches='tight')
plt.savefig('./figs/numbers_Herschel_allfields.png', bbox_inches='tight')